# 06 Lexicographic Closure

This notebook defines a method to calculate if a statement (a twiddle statement) is entailed by the lexicographic closure of a knowledge base.

In [30]:
import os
import sys


sys.path.append(os.path.join('..', 'common'))

from ranked_models import statement_ranking
from util import entails, print_knowledge_base, materialized

from itertools import combinations

from datatypes import Formula, Bot, Top, KnowledgeBase, Normally, Literal, Atom
from typing import Set


In [31]:
bot = Bot()            # Falsum
top = Top()            # Verum
m = Literal(Atom('m')) # mamalian red blood cells
v = Literal(Atom('v')) # vertebrate red blood cells
a = Literal(Atom('a')) # avian red blood cells
c = Literal(Atom('c')) # cell membrane
n = Literal(Atom('n')) # nucleus
s = Literal(Atom('s')) # mammalian sickle cells
b = Literal(Atom('b')) # bioconcave shape

Lexicographic closure works by, instead of deleting statements, weakening levels. This weakening can be done by two ways.

Either weaken the materialized version of the rank to a DNF(-like) or CNF(-like) structure.

In [32]:
def weaken_to_dnf(formulas: Set[Formula], by=0) -> Set[Formula]:
    if by <= 0:
        return formulas
    elif by >= len(formulas):
        return {Formula(Top())}
    disjuncts = combinations(formulas, by)
    formula = Bot()
    for disjunct in disjuncts:
        inner_formula = Top()
        for d in disjunct:
            inner_formula = inner_formula & d
        formula = formula | inner_formula
    return {formula}


In [33]:
def weaken_to_cnf(formulas: Set[Formula], to=0) -> Set[Formula]:
    if to <= 0:
        return formulas
    elif to > len(formulas):
        return {Formula(Top())}
    conjuncts = combinations(formulas, to)
    formulas = set()
    for conjunct in conjuncts:
        formula = Bot()
        for c in conjunct:
            formula = formula | c
        formulas.add(formula)
    return formulas

Let us compare the two approaches:

In [34]:

K={-(m >> v) / bot, -(a >> v) / bot, v / c, v / n, m / -n, -(s >> m) / bot, m / b, s / -b}
rank = statement_ranking(K)
print_knowledge_base(K)

{ m |~ b, m → v, v |~ c, m |~ ¬n, s → m, a → v, s |~ ¬b, v |~ n }


`weaken_to_dnf(R, N)` can be read as: reduce the clause by `N` statements (twiddle statements between commas) and combine them conjunctivly. If `N` $\ge$ $|K_r|$ then $\top$ remains.

In [35]:
r0 = materialized(rank[0])
print(weaken_to_dnf(r0, 0))
print(weaken_to_dnf(r0, 1))
print(weaken_to_dnf(r0, 2))

{v → n, v → c}
{v → n ∨ v → c}
{⊤}


In contrast. `weaken_to_cnf(R, N)` can be read as: For each new clause select `N` statements and combine them disjunctivly.

In [36]:
print(weaken_to_cnf(r0, 1))
print(weaken_to_cnf(r0, 2))
print(weaken_to_cnf(r0, 3))

{v → n, v → c}
{v → n ∨ v → c}
{⊤}


As you can see these approaches produce the same result with a set of two statements. The difference can be seen better with more clauses:

DNF(-like):

In [37]:
rinf = materialized(rank[float('inf')])
print(weaken_to_dnf(rinf, 0))
print(weaken_to_dnf(rinf, 1))
print(weaken_to_dnf(rinf, 2))
print(weaken_to_dnf(rinf, 3))

{a → v, m → v, s → m}
{a → v ∨ m → v ∨ s → m}
{a → v ∧ m → v ∨ a → v ∧ s → m ∨ m → v ∧ s → m}
{⊤}


CNF(-like):

In [38]:
print(weaken_to_cnf(rinf, 1))
print(weaken_to_cnf(rinf, 2))
print(weaken_to_cnf(rinf, 3))
print(weaken_to_cnf(rinf, 4))

{a → v, m → v, s → m}
{m → v ∨ s → m, a → v ∨ m → v, a → v ∨ s → m}
{a → v ∨ m → v ∨ s → m}
{⊤}


The lexicographic closure then uses these methods to reduce the ranks until the antecedent of the statement is no longer entailed to be false.

In [39]:
def lexicographic_closure_dnf(knowledge_base: KnowledgeBase, statement: Normally):
    rank = statement_ranking(knowledge_base)
    i = 0
    r = len(rank) - (float('inf') in rank)
    if r == 0:
        t = materialized(rank[float('inf')])
        rki = -Top() / Bot()
    else:
        while True:
            t = materialized({statement for k in (*range(i+1, r), float('inf')) for statement in rank[k]})
            j = 0
            n = len(rank[i])
            while True:
                rki = weaken_to_dnf(materialized(rank[i]), j)
                j+=1
                if j > n or not entails(t | rki, -statement.left):
                    break
            i += 1
            if i >= r or not entails(t | rki, -statement.left):
               break
    return entails(t | rki, statement.materialize())


In [40]:
lexicographic_closure_dnf(K, s / -n)

True

In [41]:
lexicographic_closure_dnf(K, m / -b)

False

In [42]:
def lexicographic_closure_cnf(knowledge_base: KnowledgeBase, statement: Normally):
    rank = statement_ranking(knowledge_base)
    i = 0
    r = len(rank) - (float('inf') in rank)
    if r == 0:
        t = materialized(rank[float('inf')])
        rki = -Top() / Bot()
    else:
        while True:
            t = materialized({statement for k in (*range(i+1, r), float('inf')) for statement in rank[k]})
            j = 0
            n = len(rank[i])
            while True:
                rki = weaken_to_cnf(materialized(rank[i]), j+1)
                j+=1
                if j > n or not entails(t | rki, -statement.left):
                    break
            i += 1
            if i >= r or not entails(t | rki, -statement.left):
               break
    return entails(t | rki, statement.materialize())


In [43]:
lexicographic_closure_cnf(K, s/-n)

True

In [44]:
lexicographic_closure_cnf(K, m/-b)

False